Read CSV

In [1]:
import pandas as pd
import numpy as np

unaggregated_ohlc = pd.read_csv("data/BTCUSDT.csv.gz")
unaggregated_ohlc['timestamp'] = pd.to_datetime(
    unaggregated_ohlc['Date'], unit='ms')
unaggregated_ohlc.drop("Date", axis=1)
df = unaggregated_ohlc.resample('15min', on="timestamp").agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'BaseVolume': 'sum',
    'QuoteVolume': 'sum'})

df.dropna()
df.head()

,Open,High,Low,Close,BaseVolume,QuoteVolume
timestamp,,,,,,
2018-01-01 00:00:00,13715.65,13715.65,13400.01,13520.00,247.863049,2.429390e+06
2018-01-01 00:15:00,13533.75,13550.87,13402.00,13500.00,195.173346,1.644608e+06
2018-01-01 00:30:00,13500.00,13545.37,13450.00,13493.75,168.960339,1.590488e+06
2018-01-01 00:45:00,13494.65,13690.87,13450.00,13543.87,273.407135,2.533454e+06
2018-01-01 01:00:00,13528.99,13571.74,13402.28,13468.00,145.954960,1.226711e+06


Calculate Indicators

In [2]:
import signals
from ta import momentum, volume, volatility;
import importlib

df["EffectivePrice"] = df['QuoteVolume'] / df['BaseVolume']
df["rsi7"] = momentum.rsi(df['Close'], 7) / 100
df["rsi14"] = momentum.rsi(df['Close'], 14) / 100
df["rsi21"] = momentum.rsi(df['Close'], 21) / 100
df["mfi7"] = volume.money_flow_index(
    high=df['High'], low=df['Low'], close=df['Close'], volume=df['QuoteVolume'], window=7) / 100
df["mfi14"] = volume.money_flow_index(
    high=df['High'], low=df['Low'], close=df['Close'], volume=df['QuoteVolume'], window=14) / 100
df["mfi21"] = volume.money_flow_index(
    high=df['High'], low=df['Low'], close=df['Close'], volume=df['QuoteVolume'], window=21) / 100
obv = volume.on_balance_volume(close=df["Close"], volume=df["QuoteVolume"]) / 100
df["obv7"] = momentum.rsi(obv, 7) / 100
df["obv14"] = momentum.rsi(obv, 14) / 100
df["obv21"] = momentum.rsi(obv, 21) / 100
df["willy14"] = momentum.williams_r(
    high=df["High"], low=df["Low"], close=df["Close"], lbp=14) / 100 + 1



Calculate Signals

In [3]:
importlib.reload(signals)

minimum_deviation = 0.01
zigzag = signals.zigzag(df, 'High', 'Low', minimum_deviation)

data_size = len(df)

last_index = min(zigzag.eq(-1).idxmax(), zigzag.eq(+1).idxmax())
final_index = max(zigzag.where(
    zigzag == +1).last_valid_index(), zigzag.where(zigzag == -1).last_valid_index())
df["Long"] = [0] * data_size
df["Short"] = [0] * data_size

long_col = df.columns.get_loc("Long")
short_col = df.columns.get_loc("Short")

#for i in range(last_index, final_index + 1):
#    if zigzag[i] != 0:
#        df.iloc[last_index:i, long_col] = df.iloc[last_index:i,
#                                                 long_col] / (i - last_index)
#        df.iloc[last_index:i, short_col] = df.iloc[last_index:i, short_col] / \
#            (i - last_index)
#        if zigzag[i] == +1:
#            df.iloc[last_index:i, short_col] = 1 - \
#                df.iloc[last_index:i, short_col]
#        elif zigzag[i] == -1:
#            df.iloc[last_index:i, long_col] = 1 - \
#                df.iloc[last_index:i, long_col]
#        last_index = i
#
#    df.iloc[i, long_col] = i - last_index
#    df.iloc[i, short_col] = i - last_index

for i in range(data_size):
    if zigzag[i] == +1:
        df.iloc[i, long_col] = 1
    elif zigzag[i] == -1:
        df.iloc[i, short_col] = 1


Create datasets

In [14]:
import os
import tensorflow as tf
from tensorflow import keras

df = df.astype(np.float64)

initial_skip_size = 100

train_size = round(data_size * 0.9 - initial_skip_size)
validation_size = round(data_size * 0.05 - initial_skip_size)
test_size = (data_size - initial_skip_size) - train_size - validation_size

x_columns = ["rsi7", "rsi14", "rsi21", "mfi7", "mfi14",
             "mfi21", "obv7", "obv14", "obv21", "willy14"]
y_columns = ["Long", "Short"]
window_size = 50
n_features = len(x_columns)

generator = keras.preprocessing.sequence.TimeseriesGenerator(
    df[x_columns][initial_skip_size:train_size].to_numpy(),
    df.shift(1)[y_columns][initial_skip_size:train_size].to_numpy(),
    length=window_size, start_index=1, sampling_rate=1, batch_size=10)

generator[0]


(array([[[ 62.14188286,  62.11399364,  59.94678192, ...,  47.3481062 ,
           49.43822497, -34.34412816],
         [ 57.69961781,  59.96885873,  58.60642007, ...,  43.04606545,
           46.32425338, -39.99676761],
         [ 51.85547969,  57.09914601,  56.8042922 , ...,  38.76640375,
           43.10768278, -47.63530576],
         ...,
         [ 73.60787943,  66.01922406,  61.70632697, ...,  55.80712872,
           54.88611877, -12.09466728],
         [ 69.72733113,  64.18550618,  60.58138336, ...,  49.37804993,
           50.45530805, -19.19027431],
         [ 77.12646239,  69.05388863,  64.18687425, ...,  53.80963338,
           53.50318185,  -5.23752969]],
 
        [[ 57.69961781,  59.96885873,  58.60642007, ...,  43.04606545,
           46.32425338, -39.99676761],
         [ 51.85547969,  57.09914601,  56.8042922 , ...,  38.76640375,
           43.10768278, -47.63530576],
         [ 47.11554019,  54.69924699,  55.28462538, ...,  33.6180306 ,
           39.03682836, -54.1384

Create a model and train

In [15]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'



model = keras.Sequential()

input_layer = keras.layers.Input((window_size, n_features))

rsi7_layer = keras.layers.Lambda(
    lambda x: x[:window_size, 0], output_shape=(window_size,))(input_layer)
rsi14_layer = keras.layers.Lambda(
    lambda x: x[:window_size, 1], output_shape=(window_size,))(input_layer)
rsi21_layer = keras.layers.Lambda(
    lambda x: x[:window_size, 2], output_shape=(window_size,))(input_layer)
mfi7_layer = keras.layers.Lambda(
    lambda x: x[:window_size, 3], output_shape=(window_size,))(input_layer)
mfi14_layer = keras.layers.Lambda(
    lambda x: x[:window_size, 4], output_shape=(window_size,))(input_layer)
mfi21_layer = keras.layers.Lambda(
    lambda x: x[:window_size, 5], output_shape=(window_size,))(input_layer)
obv7_layer = keras.layers.Lambda(
    lambda x: x[:window_size, 6], output_shape=(window_size,))(input_layer)
obv14_layer = keras.layers.Lambda(
    lambda x: x[:window_size, 7], output_shape=(window_size,))(input_layer)
obv21_layer = keras.layers.Lambda(
    lambda x: x[:window_size, 8], output_shape=(window_size,))(input_layer)
willy14_layer = keras.layers.Lambda(
    lambda x: x[:window_size, 9], output_shape=(window_size,))(input_layer)

processing_layer_count = 16

rsi7_dense_layer = keras.layers.Dense(
    processing_layer_count, activation="sigmoid")(rsi7_layer)
rsi14_dense_layer = keras.layers.Dense(
    processing_layer_count, activation="sigmoid")(rsi14_layer)
rsi21_dense_layer = keras.layers.Dense(
    processing_layer_count, activation="sigmoid")(rsi21_layer)
mfi7_dense_layer = keras.layers.Dense(
    processing_layer_count, activation="sigmoid")(mfi7_layer)
mfi14_dense_layer = keras.layers.Dense(
    processing_layer_count, activation="sigmoid")(mfi14_layer)
mfi21_dense_layer = keras.layers.Dense(
    processing_layer_count, activation="sigmoid")(mfi21_layer)
obv7_dense_layer = keras.layers.Dense(
    processing_layer_count, activation="sigmoid")(obv7_layer)
obv14_dense_layer = keras.layers.Dense(
    processing_layer_count, activation="sigmoid")(obv14_layer)
obv21_dense_layer = keras.layers.Dense(
    processing_layer_count, activation="sigmoid")(obv21_layer)
willy14_dense_layer = keras.layers.Dense(
    processing_layer_count, activation="sigmoid")(willy14_layer)


reduction_layer = keras.layers.Concatenate()(
    [
        rsi7_dense_layer,
        rsi14_dense_layer,
        rsi21_dense_layer,
        mfi7_dense_layer,
        mfi14_dense_layer,
        mfi21_dense_layer,
        obv7_dense_layer,
        obv14_dense_layer,
        obv21_dense_layer,
        willy14_dense_layer
    ]
)

decision_layer = keras.layers.Dense(
    8, activation="sigmoid")(reduction_layer)

output_layer = keras.layers.Dense(2, activation="sigmoid")(decision_layer)

model = keras.Model(input_layer, output_layer)

opt = keras.optimizers.Adam(learning_rate=10)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])



Train

In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 50, 10)]     0           []                               
                                                                                                  
 lambda_10 (Lambda)             (None, 10)           0           ['input_2[0][0]']                
                                                                                                  
 lambda_11 (Lambda)             (None, 10)           0           ['input_2[0][0]']                
                                                                                                  
 lambda_12 (Lambda)             (None, 10)           0           ['input_2[0][0]']                
                                                                                            

In [17]:
model.fit(generator, epochs=10)

Epoch 1/10
 3515/12873 [=======>......................] - ETA: 18s - loss: nan - accuracy: 0.9815

KeyboardInterrupt: 

In [13]:
generator = keras.preprocessing.sequence.TimeseriesGenerator(
    df[x_columns][train_size: train_size + validation_size].to_numpy(), df.shift(
        1)[y_columns][train_size: train_size + validation_size].to_numpy(),
    length=window_size, start_index=0, sampling_rate=1, batch_size=1)

X, Y = generator[0]
model.predict(X)

array([[nan, nan]], dtype=float32)